In [ ]:
import os
import json

In [ ]:
with open('base_alunos.json', 'r') as f:
    base_alunos = json.load(f)
    
base_alunos.sort( key = lambda e: e['id'] )
base_alunos

In [ ]:
with open('ground_truth_associacao.json', 'r') as f:
    ground_truth_associacao = json.load(f)
    
#ground_truth_associacao = { int(x): ground_truth_associacao[x] for x in ground_truth_associacao}
ground_truth_associacao

# Início da Avaliação de OCR de Matrículas com Detect Document Plus Sorting

In [ ]:
directory = 'ocr/matricula/vision_document_detection_plus_sorting'
import os
resultados_ocr_matricula_com_ddocplus = []
for transcription_file in os.listdir(directory):
    if not transcription_file.endswith('txt'):
        continue
    with open(f'{directory}/{transcription_file}', 'r') as f:
        transcription = ''.join(f.readlines())
        transcription = transcription.replace('\n', '')
        transcription = transcription.replace(' ', '')
        transcription = transcription.replace('MATRICULA', '')
        transcription = transcription.replace('MATRÍCULA', '')
        transcription = transcription.replace('o', '0')
        transcription = transcription.replace('O', '0')
        transcription = transcription.replace('I', '1')
    resultados_ocr_matricula_com_ddocplus.append( (int(transcription_file.split('_')[0]),transcription) )
    
resultados_ocr_matricula_com_ddocplus.sort()
resultados_ocr_matricula_com_ddocplus = { str(i):j for i,j in resultados_ocr_matricula_com_ddocplus}
resultados_ocr_matricula_com_ddocplus

In [ ]:
ground_truth_matricula = {}
for x in ground_truth_associacao:
    matricula = list(filter(lambda e: e['id'] == ground_truth_associacao[x], base_alunos))[0]['matricula']
    ground_truth_matricula[x] = matricula
    
ground_truth_matricula


In [ ]:
#precisamos comparar resultados_ocr_matricula_com_tesseract com ground_truth_matricula
for x in resultados_ocr_matricula_com_ddocplus:
    print("%35s %35s" %(resultados_ocr_matricula_com_ddocplus[x], ground_truth_matricula[x]))

In [ ]:
#avaliar quantos são absolutamente iguais
comparativo = {i: int(resultados_ocr_matricula_com_ddocplus[str(i)]==ground_truth_matricula[str(i)]) for i in resultados_ocr_matricula_com_ddocplus}
comparativo

In [ ]:
#acuracia absoluta
acuracia_absoluta = sum(comparativo.values())/len(comparativo)
acuracia_absoluta

In [ ]:
!pip install pybind11
!pip install fastwer

In [ ]:
import fastwer

In [ ]:
#calcular os CER's do OCR na matrícula
CERs = [ fastwer.score_sent(resultados_ocr_matricula_com_ddocplus[x], ground_truth_matricula[x], char_level=True) 
         for x in resultados_ocr_matricula_com_ddocplus ]

from math import isnan
for i,_ in enumerate(CERs):
    if isnan(CERs[i]):
        CERs[i] = float('inf')
CERs.sort()
CERs

In [ ]:
#calcular o pior, o melhor, o médio, o mediano e os quartis de CER
#note que alguns CERs deram infinito pois o cálculo do CER envolve dividir por len(ground_truth), que no caso de alguns formulários, é 0
from statistics import mean
from statistics import median
from statistics import quantiles
primeiro_quartil, mediana, segundo_quartil = quantiles(CERs, n=4, method='inclusive')
print('Melhor CER:', min(CERs))
print('1º quartil CER:',primeiro_quartil)
print('CER médio:', mean(CERs))
print('CER mediano:',mediana)
print('2º quartil CER:',segundo_quartil)
print('Pior CER:', max(CERs))

In [ ]:
#comparar cada valor de resultados_ocr_matricula_com_dtext com cada valor da base_alunos
associacoes = {}
for x in resultados_ocr_matricula_com_ddocplus:
    associacoes[x] = min( [ (aluno['id'], 
                        fastwer.score_sent(resultados_ocr_matricula_com_ddocplus[x], aluno['matricula'], char_level=True))
                        for aluno in base_alunos ],
                         key = lambda e: e[1])[0]
associacoes#é o aluno predicted, para cada formulário

In [ ]:
json.dump( associacoes, open( "modelo_associacoes.json", 'w' ) )

In [ ]:
#agora queremos comparar quantas vezes predicted == correto
#ou seja, precisamos contar quantas vezes associacoes[i] == ground_truth_associacao[i]
avaliacao_associacoes = [ associacoes[i]==ground_truth_associacao[i] for i in associacoes]
avaliacao_associacoes

In [ ]:
acuracia_associacao = len( list(filter(None, avaliacao_associacoes)) )/ len(avaliacao_associacoes)
acuracia_associacao*100

# Início da Avaliação de OCR de Nomes com Detect Document Plus Sorting

In [ ]:
directory = 'ocr/nome/vision_document_detection_plus_sorting'
import os
resultados_ocr_nome_com_ddocplus = []
for transcription_file in os.listdir(directory):
    if not transcription_file.endswith('txt'):
        continue
    with open(f'{directory}/{transcription_file}', 'r') as f:
        transcription = ''.join(f.readlines())
        transcription = transcription.replace('\n', '')
        transcription = transcription.replace(' ', '')
        transcription = transcription.replace('NOME', '')
        transcription = transcription.replace('0', 'O')
        transcription = transcription.replace('1', 'I')
        transcription = transcription.upper()
    resultados_ocr_nome_com_ddocplus.append( (int(transcription_file.split('_')[0]),transcription) )
    
resultados_ocr_nome_com_ddocplus.sort()
resultados_ocr_nome_com_ddocplus = { str(i):j for i,j in resultados_ocr_nome_com_ddocplus}
resultados_ocr_nome_com_ddocplus

In [ ]:
ground_truth_nome = {}
for x in ground_truth_associacao:
    nome = list(filter(lambda e: e['id'] == ground_truth_associacao[x], base_alunos))[0]['nome']
    ground_truth_nome[x] = nome
    
ground_truth_nome

In [ ]:
#precisamos comparar resultados_ocr_nome_com_dtext com ground_truth_nome
for x in resultados_ocr_nome_com_ddocplus:
    print("%50s %50s" %(resultados_ocr_nome_com_ddocplus[x], ground_truth_nome[x]))

In [ ]:
#avaliar quantos são absolutamente iguais
comparativo = {i: int(resultados_ocr_nome_com_ddocplus[str(i)]==ground_truth_nome[str(i)]) for i in resultados_ocr_nome_com_ddocplus}
comparativo

In [ ]:
#acuracia absoluta
acuracia_absoluta = sum(comparativo.values())/len(comparativo)
acuracia_absoluta

In [ ]:
!pip install pybind11
!pip install fastwer
import fastwer

In [ ]:
#calcular os CER's do OCR nos nomes
CERs = [ fastwer.score_sent(resultados_ocr_nome_com_ddocplus[x], ground_truth_nome[x], char_level=True) 
         for x in resultados_ocr_nome_com_ddocplus ]
CERs.sort()
CERs

In [ ]:
#calcular o pior, o melhor, o médio, o mediano e os quartis de CER
#note que alguns CERs deram infinito pois o cálculo do CER envolve dividir por len(ground_truth), que no caso de alguns formulários, é 0
from statistics import mean
from statistics import median
from statistics import quantiles
primeiro_quartil, mediana, segundo_quartil = quantiles(CERs, n=4, method='inclusive')
print('Melhor CER:', min(CERs))
print('1º quartil CER:',primeiro_quartil)
print('CER médio:', mean(CERs))
print('CER mediano:',mediana)
print('2º quartil CER:',segundo_quartil)
print('Pior CER:', max(CERs))

In [ ]:
#comparar cada valor de resultados_ocr_nome_com_tesseract com cada valor da base_alunos
associacoes = {}
for x in resultados_ocr_nome_com_ddocplus:
    associacoes[x] = min( [ (aluno['id'], 
                        fastwer.score_sent(resultados_ocr_nome_com_ddocplus[x], aluno['nome'], char_level=True))
                        for aluno in base_alunos ],
                         key = lambda e: e[1])[0]
associacoes#é o aluno predicted, para cada formulário

In [ ]:
json.dump( associacoes, open( "modelo_associacoes.json", 'w' ) )

In [ ]:
#agora queremos comparar quantas vezes predicted == correto
#ou seja, precisamos contar quantas vezes associacoes[i] == ground_truth_associacao[i]
avaliacao_associacoes = [ associacoes[i]==ground_truth_associacao[i] for i in associacoes]
avaliacao_associacoes

In [ ]:
acuracia_associacao = len( list(filter(None, avaliacao_associacoes)) )/ len(avaliacao_associacoes)
acuracia_associacao*100

In [ ]:
base_alunos

In [ ]:
directory = '/home/vinicius/Documents/tcc/teste1/jsons_predictions'

obj = {}
for x in resultados_ocr_matricula_com_ddocplus:
    #procurar em modelo_associacoes, o cara com a chave x
    #o valor dele é o id de aluno a ser buscado na base de alunos
    id_aluno_predicted = associacoes[x]
    #buscar este id na base_alunos e retornar o objeto de dados correspondente
    dados_aluno = list(filter(lambda e:e['id']==id_aluno_predicted, base_alunos))[0]
    obj[x] = dados_aluno


for x in resultados_ocr_nome_com_ddocplus:
    json.dump( obj[x], open( f'{directory}/{x}.json', 'w' ) )

In [ ]:
obj